# Decorators

- A decorator is a wrapper that we can place around a function that changes that function's behavior. We can modify the inputs and outputs or even change the behaviour of the function itself.

In [ ]:
@double_args
def multiply(a, b):
    return a * b

- `double_args` is a decorator that multiplies every argument by two before passing them to the decorated function
- Decorators are just functions that takes a function as an argument and returns a modified version of that function.

In [1]:
def multiply(a, b):
    return a * b

def double_args(func):
    def wrapper(a, b):
        # call the passed in function, but double each argument
        return func(a*2, b*2)
    return wrapper

In [3]:
new_multiply = double_args(multiply)
new_multiply(1,5)

20

In [4]:
multiply = double_args(multiply)
multiply(1,5)

20

- We can do this because python stores the **original multiply function in the new function's closure**

In [5]:
@double_args
def multiply(a, b):
    return a * b

- **@double_args** on the line above the multiply function. This is just Python convenience for saying "multiply" equals the value returned by calling `double_args` with "multiply" as the only argument.

###  Defining a decorator
- a decorator that prints a "before" message before the decorated function is called and prints an "after" message after the decorated function is called.


In [6]:
def print_before_and_after(func):
  def wrapper(*args):
    print('Before {}'.format(func.__name__))
    # Call the function being decorated with *args
    func(*args)
    print('After {}'.format(func.__name__))
  # Return the nested function
  return wrapper

@print_before_and_after
def multiply(a, b):
  print(a * b)

multiply(5, 10)

Before multiply
50
After multiply


# Time a function

- The timer() decorator runs the decorated function and then prints how long it took for the function to run.
- `wrapper()` takes any number of positional and keyword arguments so that it can be used to decorate any function. The first thing this new function would do is record the time that it was called with the `time()` function.
- Then wrapper() gets the result of calling the decorated function. After calling the decorated function, wrapper() checks the time again and prints the time how long it took to run the decorated function.
- After that return the value that the decorated function calculated.

In [8]:
import time

def timer(func):
    """A decorator that prints how long a function took to run"""
    # define the wrapper function to return
    def wrapper(*args, **kwargs):
        # when wrapper() is called, get the current time
        t_start = time.time()
        # call the decorated function and store the result
        result = func(*args, **kwargs)
        # get the total time it took to run,and print it
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        return result
    return wrapper

## Using timer()

In [11]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)
    
sleep_n_seconds(5)

sleep_n_seconds(10)

sleep_n_seconds took 5.0099756717681885s
sleep_n_seconds took 10.010277509689331s


## Memoize
- Memoizing is the process of storing the results of a function so that the next time the function is called with the same arguments; we can just look up the answer.
- Start by setting up a dictionary that will map arguments to results.
- Then, as usual create a `wrapper` to  be the new decorated function that this decorator returns
- When the new function gets called, it checks to see whether it has seen these arguments before. If haven't then its send to  the decorated function and store the result in the "cache" dictionary
- Next time the function is called with the same arguments, the return value will already be in the dictionary

In [22]:
def memoize(func):
    """store the results of the decorated function for fast lookup"""
    # store results in a dict that maps arguments to results
    cache = {}
    # define the wrapper function to return
    def wrapper(*args, **kwargs):
        # if these arguments haven't been seen before
        print(args, kwargs)
        if (args) not in cache:
            # call func() and store the result
            cache[(args)] = func(*args, **kwargs)
            print(cache)
        return cache[(args)]
    return wrapper

In [23]:
@memoize
def slow_function(a, b):
    print('sleeping........')
    time.sleep(5)
    return a + b
slow_function(3, 4)

(3, 4) {}
sleeping........
{(3, 4): 7}


7

In [24]:
slow_function(3, 4)

(3, 4) {}


7

# When to use decorator
- When we want to add some common lines of code to multiple functions

## Print the return type : decorator
-  if we expect a function to return a numpy array, but it returns a list, we can get unexpected behavior. To ensure this is not what is causing the trouble, we write a decorator, **`print_return_type(), that will print out the type of the variable that gets returned from every call of any function it is decorating.`**

In [1]:
def print_return_type(func):
  # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
          func.__name__, type(result)
        ))
        return result
    # Return the decorated function
    return wrapper
  
@print_return_type
def foo(value):
    return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


##### It seems a little magical that we can reference the wrapper() function from inside the definition of wrapper() . That's just one of the many neat things about functions in Python -- any function, not just decorators.